In [2]:
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('E:\\ML\\CNN GenderD\\Train Set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 2002 images belonging to 2 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('E:\\ML\\CNN GenderD\\Test Set',
                                            target_size = (64,64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 555 images belonging to 2 classes.


In [5]:
cnn = tf.keras.models.Sequential()

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

In [7]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [9]:
cnn.add(tf.keras.layers.Flatten())

In [10]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [11]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [12]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [13]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
63/63 [==============================] - 12s 179ms/step - loss: 0.6990 - accuracy: 0.5606 - val_loss: 0.4230 - val_accuracy: 0.8252
Epoch 2/25
63/63 [==============================] - 11s 176ms/step - loss: 0.4322 - accuracy: 0.8003 - val_loss: 0.2584 - val_accuracy: 0.9027
Epoch 3/25
63/63 [==============================] - 12s 189ms/step - loss: 0.3048 - accuracy: 0.8643 - val_loss: 0.2232 - val_accuracy: 0.9135
Epoch 4/25
63/63 [==============================] - 12s 183ms/step - loss: 0.2736 - accuracy: 0.8778 - val_loss: 0.1669 - val_accuracy: 0.9351
Epoch 5/25
63/63 [==============================] - 12s 192ms/step - loss: 0.2348 - accuracy: 0.9008 - val_loss: 0.1858 - val_accuracy: 0.9315
Epoch 6/25
63/63 [==============================] - 13s 199ms/step - loss: 0.2058 - accuracy: 0.9217 - val_loss: 0.1960 - val_accuracy: 0.9297
Epoch 7/25
63/63 [==============================] - 13s 203ms/step - loss: 0.2113 - accuracy: 0.9157 - val_loss: 0.2700 - val_accuracy: 0.9009

In [37]:
from keras.preprocessing import image
test_image = image.load_img('E:/ML/trash/Test Set/Man/face_1231.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
print(result[0][0])

0.0


In [38]:
import cvlib as cv
import cv2
webcam = cv2.VideoCapture(0)  

while webcam.isOpened():
    status, frame = webcam.read()
    face, confidence = cv.detect_face(frame)
    for idx, f in enumerate(face):
        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]
        cv2.rectangle(frame, (startX,startY), (endX,endY), (0,255,0), 2)
        face_crop = np.copy(frame[startY:endY,startX:endX])
        if (face_crop.shape[0]) < 10 or (face_crop.shape[1]) < 10:
            continue
        face_crop = cv2.resize(face_crop, (64,64))
        face_crop = face_crop.astype("float") / 255.0
        face_crop = tf.keras.preprocessing.image.img_to_array(face_crop)
        face_crop = np.expand_dims(face_crop, axis=0)
        conf = cnn.predict(face_crop)
        if conf[0][0] <= 0.5:
            label = f"man"
        else:
            label = f"woman"
        Y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.putText(frame, label, (startX, Y),  cv2.FONT_HERSHEY_SIMPLEX, 0.7, (145, 155, 49), 2)
    cv2.imshow("gender detection", frame)
    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

webcam.release()
cv2.destroyAllWindows()